In [16]:
from functools import reduce
import numpy as np
import random
from scipy.linalg import expm

def generate_random_clauses(n, max_attempts=1000):
    """Generate random Exact Cover clauses with a unique satisfying assignment."""
    def count_satisfying_assignments(clauses, n):
        """Count the number of satisfying assignments for the given clauses."""
        count = 0
        for state in range(2**n):
            bits = [(state >> bit) & 1 for bit in range(n)]
            if all(sum(bits[i] for i in clause) == 1 for clause in clauses):
                count += 1
        return count

    clauses = []
    attempts = 0
    while attempts < max_attempts:
        # Generate a random clause with 3 distinct bits
        clause = tuple(sorted(random.sample(range(n), 3)))
        if clause not in clauses:  # Avoid duplicate clauses
            clauses.append(clause)
            # Count satisfying assignments
            num_satisfying = count_satisfying_assignments(clauses, n)
            if num_satisfying == 1:
                return clauses  # Found a USA instance
            elif num_satisfying == 0:
                clauses.pop()  # Reject the clause as it creates no satisfying assignments
        attempts += 1

    raise ValueError("Failed to generate a unique satisfying assignment instance.")


def exact_cover_hamiltonian(clauses, n):
    """Construct the problem Hamiltonian for Exact Cover."""
    H_P = np.zeros((2**n, 2**n))
    for clause in clauses:
        # Clause constraint: z_i + z_j + z_k = 1
        i, j, k = clause
        for state in range(2**n):
            bits = [(state >> bit) & 1 for bit in range(n)]
            if sum(bits[b] for b in (i, j, k)) != 1:
                H_P[state, state] += 1
    return H_P

def initial_hamiltonian(n):
    """Construct the initial Hamiltonian."""
    H_0 = np.zeros((2**n, 2**n))
    for i in range(2**n):
        for j in range(2**n):
            if i != j:
                H_0[i, j] = -1
    return H_0

def adiabatic_evolution(H_0, H_P, T, n, steps=100):
    """Perform adiabatic evolution."""
    psi = np.ones(2**n) / np.sqrt(2**n)  # Initial uniform superposition
#    print("Initial state (|psi(0)>):", psi)
    dt = T / steps
    for step in range(steps):
        t = step * dt
        H_t = (1 - t / T) * H_0 + (t / T) * H_P
        U_t = expm(-1j * H_t * dt)
        psi = U_t @ psi
    return psi

def measure_state(psi, n):
    """Simulate measurement of the quantum state."""
    probabilities = np.abs(psi)**2
    print("Final state probabilities:", probabilities)
    print("Sum of probabilities (should be ~1):", np.sum(probabilities))
    measurement = np.random.choice(2**n, p=probabilities)
    return [(measurement >> bit) & 1 for bit in range(n)]

def measure_state_multiple(psi, n, num_samples=100):
    """Simulate multiple measurements of the quantum state."""
    probabilities = np.abs(psi)**2
    results = [np.random.choice(2**n, p=probabilities) for _ in range(num_samples)]
    decoded_results = [[(result >> bit) & 1 for bit in range(n)] for result in results]
    return decoded_results

def get_most_common_solution(measured_solutions):
    """Find the most common solution using a reduce operation with a lambda."""
    # Reduce the list into a frequency dictionary
    frequency_dict = reduce(
        lambda acc, solution: {**acc, tuple(solution): acc.get(tuple(solution), 0) + 1},
        measured_solutions,
        {}
    )

    # Find the most common solution
    most_common_solution = max(frequency_dict, key=frequency_dict.get)
    count = frequency_dict[most_common_solution]
    return list(most_common_solution), count

def brute_force_exact_cover(clauses, n):
    """Classical brute-force algorithm for Exact Cover."""
    solutions = []
    for state in range(2**n):
        bits = [(state >> bit) & 1 for bit in range(n)]
        valid = all(sum(bits[i] for i in clause) == 1 for clause in clauses)
        if valid:
            solutions.append(bits)
    return solutions

# Define the problem
n = 4  # Number of bits
#clauses = [(0, 1, 2), (1, 2, 3)]  # Example Exact Cover clauses
random_clauses = generate_random_clauses(n)
T = 100  # Total evolution time

# Quantum adiabatic simulation
H_0 = initial_hamiltonian(n)
#print("Initial Hamiltonian (H_0):\n", H_0)
H_P = exact_cover_hamiltonian(clauses, n)
#print("Problem Hamiltonian (H_P):\n", H_P)
psi_final = adiabatic_evolution(H_0, H_P, T, n)
quantum_solution = measure_state(psi_final, n)

# Classical brute-force solution
classical_solutions = brute_force_exact_cover(clauses, n)

# Print results
print("Quantum-measured solution:", quantum_solution)
print("Classical brute-force solutions:", classical_solutions)

# Measure the quantum state multiple times
quantum_solutions = measure_state_multiple(psi_final, n)

# Find the most common solution
most_common_solution, count = get_most_common_solution(quantum_solutions)

# Output results
#print("Quantum-measured solutions (multiple):", quantum_solutions)
print(f"Most common solution: {most_common_solution} (appeared {count} times)")


Final state probabilities: [3.21814704e-05 2.68862216e-04 3.32720519e-01 2.68862216e-04
 3.32720519e-01 2.68862216e-04 3.21814704e-05 3.21814704e-05
 2.68862216e-04 3.32720519e-01 2.68862216e-04 3.21814704e-05
 2.68862216e-04 3.21814704e-05 3.21814704e-05 3.21814704e-05]
Sum of probabilities (should be ~1): 0.9999999999999858
Quantum-measured solution: [1, 0, 0, 1]
Classical brute-force solutions: [[0, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 1]]
Most common solution: [0, 0, 1, 0] (appeared 41 times)


In [3]:
def test_classical_exact_cover():
    """Test the classical brute-force solution with a simple example."""
    n = 3
    clauses = [(0, 1, 2)]  # Single clause
    expected_solutions = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    
    classical_solutions = brute_force_exact_cover(clauses, n)
    
    assert sorted(classical_solutions) == sorted(expected_solutions), \
        f"Test failed. Got {classical_solutions}, expected {expected_solutions}"
    print("Test passed for classical brute-force algorithm!")

# Run the test
test_classical_exact_cover()

Test passed for classical brute-force algorithm!
